In [138]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import time


In [139]:
# Your path to the webdriver might be different
executable_path = r'C:\Users\liu12\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(executable_path=executable_path)

url = 'https://www.loblaws.ca/food/fruits-vegetables/c/28000?navid=flyout-L2-fruits-vegetables'
driver.get(url)


C:\Users\liu12\AppData\Local\Temp\ipykernel_8648\337350798.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path)


In [140]:
# Scroll to "Load More" button and click until it no longer exists
while True:
    try:
        load_more_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.primary-button.primary-button--load-more-button')))
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        WebDriverWait(driver, 20).until(EC.staleness_of(load_more_button)) # Wait for new data to load
    except Exception as e:
        print(str(e))
        break

Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0094A813+48355]
	(No symbol) [0x008DC4B1]
	(No symbol) [0x007E5358]
	(No symbol) [0x008109A5]
	(No symbol) [0x00810B3B]
	(No symbol) [0x0083E232]
	(No symbol) [0x0082A784]
	(No symbol) [0x0083C922]
	(No symbol) [0x0082A536]
	(No symbol) [0x008082DC]
	(No symbol) [0x008093DD]
	GetHandleVerifier [0x00BAAABD+2539405]
	GetHandleVerifier [0x00BEA78F+2800735]
	GetHandleVerifier [0x00BE456C+2775612]
	GetHandleVerifier [0x009D51E0+616112]
	(No symbol) [0x008E5F8C]
	(No symbol) [0x008E2328]
	(No symbol) [0x008E240B]
	(No symbol) [0x008D4FF7]
	BaseThreadInitThunk [0x76777D59+25]
	RtlInitializeExceptionChain [0x7791B74B+107]
	RtlClearBits [0x7791B6CF+191]



In [141]:
# Now that we're done loading more results, create the soup object
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the groceries with BeautifulSoup
groceries = soup.find_all(class_='product-tracking')

In [142]:
grocery_list = []
batch_size = 20  # number of items to process at once

for i in range(0, len(groceries), batch_size):
    batch = groceries[i:i + batch_size]
    
    for grocery in batch:
        # The try/except blocks will prevent the script from crashing if it can't find an element
        try:
            brand = grocery.find(class_='product-name__item product-name__item--brand').text
        except:
            brand = "Unknown"
        
        try:
            product = grocery.find(class_='product-name__item product-name__item--name').text
        except:
            product = "Unknown"
        
        try:
            price_text = grocery.find(class_='price__value selling-price-list__item__price selling-price-list__item__price--now-price__value').text
            price = float(re.search(r'[\d.]+', price_text).group())
        except:
            price = 0.0

        grocery_dict = {'product brand': brand, "product name": product, "price": price}
        grocery_list.append(grocery_dict)
    
    time.sleep(2)  # add delay after processing each batch

# Now grocery_list should contain your scraped data
print(grocery_list)


[{'product brand': 'Unknown', 'product name': 'Bananas, Bunch', 'price': 1.75}, {'product brand': 'Unknown', 'product name': 'Strawberries 1LB', 'price': 4.99}, {'product brand': 'Unknown', 'product name': 'English Cucumber', 'price': 0.99}, {'product brand': 'Unknown', 'product name': 'Red Seedless Watermelon', 'price': 7.99}, {'product brand': 'Unknown', 'product name': 'Green Onion', 'price': 1.49}, {'product brand': 'Unknown', 'product name': 'Red Cherries', 'price': 13.71}, {'product brand': 'Unknown', 'product name': 'Tomato On The Vine Red (1 Bunch)', 'price': 1.55}, {'product brand': 'Unknown', 'product name': 'Asparagus', 'price': 5.19}, {'product brand': 'Unknown', 'product name': 'Tomato Beefsteak  Red', 'price': 1.18}, {'product brand': 'Unknown', 'product name': 'Lettuce Iceberg', 'price': 3.99}, {'product brand': 'Unknown', 'product name': 'Avocados Bag', 'price': 4.99}, {'product brand': 'Unknown', 'product name': 'Lemon', 'price': 0.99}, {'product brand': "Farmer's Mark

In [143]:
# Close Driver
driver.quit()

# Convert to Pandas Dataframe

In [144]:
loblaws_df = pd.DataFrame(grocery_list)
loblaws_df.head(100)

,product brand,product name,price
0,Unknown,"Bananas, Bunch",1.75
1,Unknown,Strawberries 1LB,4.99
2,Unknown,English Cucumber,0.99
3,Unknown,Red Seedless Watermelon,7.99
4,Unknown,Green Onion,1.49
...,...,...,...
95,PC Organics,Organic Lime,6.99
96,Unknown,"Fresh Berry Bowl, Small",7.00
97,President's Choice,Creamy Golden Potatoes,3.99
98,Unknown,Cobb Salad with Cheddar Chips,9.00


In [145]:
loblaws_df['store'] = 'Toronto, Carlton Store'
loblaws_df.head()

,product brand,product name,price,store
0,Unknown,"Bananas, Bunch",1.75,"Toronto, Carlton Store"
1,Unknown,Strawberries 1LB,4.99,"Toronto, Carlton Store"
2,Unknown,English Cucumber,0.99,"Toronto, Carlton Store"
3,Unknown,Red Seedless Watermelon,7.99,"Toronto, Carlton Store"
4,Unknown,Green Onion,1.49,"Toronto, Carlton Store"


In [146]:
print(len(loblaws_df))

674


In [147]:
loblaws_df['product brand'].unique()

array(['Unknown', "Farmer's Market", 'PC Organics', "President's Choice",
       'Rooster', 'The Little Potato Company', 'Fresh Attitude',
       'Greenhouse', 'Aurora', 'Taylor Farms', 'Litehouse', 'No Name',
       'Renees', 'Paramount', 'Marzetti', 'Bolthouse', 'Gourmet Garden',
       'Tiffany Gates', 'GoodLeaf', 'Love Beets', 'Loop', '7D', 'Sunrich',
       'Eat Smart', 'Dole', 'Earthbound Farms Organic', 'Well',
       'Wholly Guacamole', 'Remedy', 'Duda Farms', 'Manns',
       'POM Wonderful', 'PC Blue Menu'], dtype=object)

In [148]:
loblaws_df.to_csv('Loblaws_Toronto_Data.csv', index=False)